In [1]:
from __future__ import print_function
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from pyvi import ViTokenizer
import re
import string
import codecs

Gọi Dữ Liệu Và Đọc Dữ Liệu Theo Từng Dòng Tương Ứng

In [2]:
#Từ điển thất vọng, hài lòng, bình thường
path_thatvong = 'camxuc/Thất Vọng.txt'
path_hailong = 'camxuc/Hài Lòng.txt'
path_binhthuong = 'camxuc/Bình Thường.txt'
path_not = 'camxuc/not.txt'


with codecs.open(path_thatvong, 'r', encoding='UTF-8') as f:
    thatvong = f.readlines()
thatvong_list = [n.replace('\n', '') for n in thatvong]

with codecs.open(path_hailong, 'r', encoding='UTF-8') as f:
    hailong = f.readlines()
hailong_list = [n.replace('\n', '') for n in hailong]

with codecs.open(path_binhthuong, 'r', encoding='UTF-8') as f:
    binhthuong = f.readlines()
binhthuong_list = [n.replace('\n', '') for n in binhthuong]

with codecs.open(path_not, 'r', encoding='UTF-8') as f:
    not_list = f.readlines()

Bỏ các ký tự tiếng việt có dấu trong dữ liệu và chuyển thành các ký tự không dấu tương ứng

In [3]:
VN_CHARS_LOWER = u'ạảãàáâậầấẩẫăắằặẳẵóòọõỏôộổỗồốơờớợởỡéèẻẹẽêếềệểễúùụủũưựữửừứíìịỉĩýỳỷỵỹđð'
VN_CHARS_UPPER = u'ẠẢÃÀÁÂẬẦẤẨẪĂẮẰẶẲẴÓÒỌÕỎÔỘỔỖỒỐƠỜỚỢỞỠÉÈẺẸẼÊẾỀỆỂỄÚÙỤỦŨƯỰỮỬỪỨÍÌỊỈĨÝỲỶỴỸÐĐ'
VN_CHARS = VN_CHARS_LOWER + VN_CHARS_UPPER
def no_marks(s):
    __INTAB = [ch for ch in VN_CHARS]
    __OUTTAB = "a"*17 + "o"*17 + "e"*11 + "u"*11 + "i"*5 + "y"*5 + "d"*2
    __OUTTAB += "A"*17 + "O"*17 + "E"*11 + "U"*11 + "I"*5 + "Y"*5 + "D"*2
    __r = re.compile("|".join(__INTAB))
    __replaces_dict = dict(zip(__INTAB, __OUTTAB))
    result = __r.sub(lambda m: __replaces_dict[m.group(0)], s)
    return result

Nhận một chuỗi văn bản đầu vào và thực hiện một loạt các bước tiền xử lý để chuẩn hóa văn bản trước khi sử dụng cho các mô hình xử lý ngôn ngữ tự nhiên

In [4]:
def normalize_text(text):

    #Remove các ký tự kéo dài: vd: đẹppppppp
    text = re.sub(r'([A-Z])\1+', lambda m: m.group(1).upper(), text, flags=re.IGNORECASE)

    # Chuyển thành chữ thường
    text = text.lower()

    #Chuẩn hóa tiếng Việt, xử lý emoj, chuẩn hóa tiếng Anh, thuật ngữ
    replace_list = {
        'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ', 'òe': 'oè', 'óe': 'oé','ỏe': 'oẻ',
        'õe': 'oẽ', 'ọe': 'oẹ', 'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ','ụy': 'uỵ', 'uả': 'ủa',
        'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
        'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
        'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
        #Quy các icon về 2 loại emoj: Hài Lòng và Thất vọng
        "👹": "thatvong", "👻": "hailong", "💃": "hailong",'🤙': ' hailong ', '👍': ' hailong ',
        "💄": "hailong", "💎": "hailong", "💩": "hailong","😕": "thatvong", "😱": "thatvong", "😸": "hailong",
        "😾": "thatvong", "🚫": "thatvong",  "🤬": "thatvong","🧚": "hailong", "🧡": "hailong",'🐶':' hailong ',
        '👎': ' thatvong ', '😣': ' thatvong ','✨': ' hailong ', '❣': ' hailong ','☀': ' hailong ',
        '♥': ' hailong ', '🤩': ' hailong ', 'like': ' hailong ', '💌': ' hailong ',
        '🤣': ' hailong ', '🖤': ' hailong ', '🤤': ' hailong ', ':(': ' thatvong ', '😢': ' thatvong ',
        '❤': ' hailong ', '😍': ' hailong ', '😘': ' hailong ', '😪': ' thatvong ', '😊': ' hailong ',
        '?': ' ? ', '😁': ' hailong ', '💖': ' hailong ', '😟': ' thatvong ', '😭': ' thatvong ',
        '💯': ' hailong ', '💗': ' hailong ', '♡': ' hailong ', '💜': ' hailong ', '🤗': ' hailong ',
        '^^': ' hailong ', '😨': ' thatvong ', '☺': ' hailong ', '💋': ' hailong ', '👌': ' hailong ',
        '😖': ' thatvong ', '😀': ' hailong ', ':((': ' thatvong ', '😡': ' thatvong ', '😠': ' thatvong ',
        '😒': ' thatvong ', '🙂': ' hailong ', '😏': ' thatvong ', '😝': ' hailong ', '😄': ' hailong ',
        '😙': ' hailong ', '😤': ' thatvong ', '😎': ' hailong ', '😆': ' hailong ', '💚': ' hailong ',
        '✌': ' hailong ', '💕': ' hailong ', '😞': ' thatvong ', '😓': ' thatvong ', '️🆗️': ' hailong ',
        '😉': ' hailong ', '😂': ' hailong ', ':v': '  hailong ', '=))': '  hailong ', '😋': ' hailong ',
        '💓': ' hailong ', '😐': ' thatvong ', ':3': ' hailong ', '😫': ' thatvong ', '😥': ' thatvong ',
        '😃': ' hailong ', '😬': ' 😬 ', '😌': ' 😌 ', '💛': ' hailong ', '🤝': ' hailong ', '🎈': ' hailong',
        '😗': ' hailong ', '🤔': ' thatvong ', '😑': ' thatvong ', '🔥': ' thatvong ', '🙏': ' thatvong ',
        '🆗': ' hailong ', '😻': ' hailong ', '💙': ' hailong ', '💟': ' hailong ',
        '😚': ' hailong ', '❌': ' thatvong ', '👏': ' hailong ', ';)': ' hailong ', '<3': ' hailong ',
        '🌝': ' hailong ',  '🌷': ' hailong ', '🌸': ' hailong ', '🌺': ' hailong ',
        '🌼': ' hailong ', '🍓': ' hailong ', '🐅': ' hailong ', '🐾': ' hailong ', '👉': ' hailong ',
        '💐': ' hailong ', '💞': ' hailong ', '💥': ' hailong ', '💪': ' hailong ',
        '💰': ' hailong ',  '😇': ' hailong ', '😛': ' hailong ', '😜': ' hailong ',
        '🙃': ' hailong ', '🤑': ' hailong ', '🤪': ' hailong ','☹': ' thatvong ',  '💀': ' thatvong ',
        '😔': ' thatvong ', '😧': ' thatvong ', '😩': ' thatvong ', '😰': ' thatvong ', '😳': ' thatvong ',
        '😵': ' thatvong ', '😶': ' thatvong ', '🙁': ' thatvong ',
        #Chuẩn hóa 1 số sentiment words/English words
        ':))': '  hailong ', ':)': ' hailong ', 'ô kêi': ' ok ', 'okie': ' ok ', ' o kê ': ' ok ',
        'okey': ' ok ', 'ôkê': ' ok ', 'oki': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
        ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ',
        '⭐': 'star ', '*': 'star ', '🌟': 'star ', '🎉': u' hailong ',
        'kg ': u' không ','not': u' không ', u' kg ': u' không ', '"k ': u' không ',' kh ':u' không ','kô':u' không ','hok':u' không ',' kp ': u' không phải ',u' kô ': u' không ', '"ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',
        'he he': ' hailong ','hehe': ' hailong ','hihi': ' hailong ', 'haha': ' hailong ', 'hjhj': ' hailong ',
        ' lol ': ' thatvong ',' cc ': ' thatvong ','cute': u' dễ thương ','huhu': ' thatvong ', ' vs ': u' với ', 'wa': ' quá ', 'wá': u' quá', 'j': u' gì ', '“': ' ',
        ' sz ': u' cỡ ', 'size': u' cỡ ', u' đx ': u' được ', 'dk': u' được ', 'dc': u' được ', 'đk': u' được ',
        'đc': u' được ','authentic': u' chuẩn chính hãng ',u' aut ': u' chuẩn chính hãng ', u' auth ': u' chuẩn chính hãng ', 'thick': u' hailong ', 'store': u' cửa hàng ',
        'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
        'sấu': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
        'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
        'ể': 'ể', 'product': 'sản phẩm', 'quality': 'chất lượng','chat':' chất ', 'excelent': 'hoàn hảo', 'bad': 'tệ','fresh': ' tươi ','sad': ' tệ ',
        'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
        'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
        'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
        'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ',u'quả ng ':u' quảng  ',
        'dep': u' đẹp ',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u'qủa': u' quả ',
        'iu': u' yêu ','fake': u' giả mạo ', 'trl': 'trả lời', '><': u' hailong ',
        ' por ': u' tệ ',' poor ': u' tệ ', 'ib':u' nhắn tin ', 'rep':u' trả lời ',u'fback':' feedback ','fedback':' feedback ',
        #dưới 3* quy về 1*, trên 3* quy về 5*
        '6 sao': ' 5star ','6 star': ' 5star ', '5star': ' 5star ','5 sao': ' 5star ','5sao': ' 5star ',
        'starstarstarstarstar': ' 5star ', '1 sao': ' 1star ', '1sao': ' 1star ','2 sao':' 1star ','2sao':' 1star ',
        '2 starstar':' 1star ','1star': ' 1star ', '0 sao': ' 1star ', '0star': ' 1star ',}

    for k, v in replace_list.items():
        text = text.replace(k, v)

    # chuyen punctuation thành space
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    text = text.translate(translator)

    text = ViTokenizer.tokenize(text)
    texts = text.split()
    len_text = len(texts)

    texts = [t.replace('_', ' ') for t in texts]
    for i in range(len_text):
        cp_text = texts[i]
        if cp_text in not_list: # Xử lý vấn đề phủ định (VD: áo này chẳng đẹp--> áo này notpos)
            numb_word = 2 if len_text - i - 1 >= 4 else len_text - i - 1

            for j in range(numb_word):
                if texts[i + j + 1] in hailong_list:
                    texts[i] = 'notpos'
                    texts[i + j + 1] = ''

                if texts[i + j + 1] in thatvong_list:
                    texts[i] = 'notnag'
                    texts[i + j + 1] = ''
        else: #Thêm feature cho những sentiment words (áo này đẹp--> áo này đẹp)
            if cp_text in hailong_list:
                texts.append('hailong')
            elif cp_text in thatvong_list:
                texts.append('thatvong')

    text = u' '.join(texts)

    #remove nốt những ký tự thừa thãi
    text = text.replace(u'"', u' ')
    text = text.replace(u'️', u'')
    text = text.replace('🏻','')
    return text

Tải dữ liệu từ các tệp tin và chuyển đổi chúng thành dạng dữ liệu phù hợp để sử dụng trong mô hình

In [5]:
class DataSource(object):
#Đọc dữ liệu từ một tệp tin được chỉ định và chia thành các mẫu dữ liệu riêng biệt.
    def _load_raw_data(self, filename, is_train=True):

        a = []
        b = []

        regex = 'train_'
        if not is_train:
            regex = 'test_'

        with open(filename, 'r', encoding="utf8") as file:
            for line in file:
                if regex in line:
                    b.append(a)
                    a = [line]
                elif line != '\n':
                    a.append(line)
        b.append(a)

        return b[1:]
#Tạo ra một từ điển từ một mẫu dữ liệu. Nó lấy một mẫu và trích xuất id, review và label từ nó
    def _create_row(self, sample, is_train=True):

        d = {}
        d['id'] = sample[0].replace('\n', '')
        review = ""
        #Đối với dữ liệu huấn luyện, nó tạo ra một từ điển có khóa 'id', 'review' và 'label'.
        if is_train:
            for clause in sample[1:-1]:
                review += clause.replace('\n', ' ')
                review = review.replace('.', ' ')

            d['label'] = int(sample[-1].replace('\n', ' '))
        #Đối với dữ liệu kiểm tra, nó chỉ tạo ra một từ điển có khóa 'id' và 'review'
        else:
            for clause in sample[1:]:
                review += clause.replace('\n', ' ')
                review = review.replace('.', ' ')


        d['review'] = review

        return d
#Tải dữ liệu từ tệp tin và chuyển đổi chúng thành danh sách các từ điển.
    def load_data(self, filename, is_train=True):

        raw_data = self._load_raw_data(filename, is_train)
        lst = []

        for row in raw_data:
            lst.append(self._create_row(row, is_train))

        return lst
#Nhận danh sách các mẫu và nhãn và chuyển đổi chúng thành dạng dữ liệu phù hợp để sử dụng trong mô hình.
    def transform_to_dataset(self, x_set,y_set):
        X, y = [], []
        #Chuẩn hóa văn bản sử dụng hàm normalize_text và tạo ra các biến đổi bổ sung bằng cách loại bỏ dấu tiếng Việt sử dụng hàm no_marks
        for document, topic in zip(list(x_set), list(y_set)):
            document = normalize_text(document)
            X.append(document.strip())
            y.append(topic)
            #Augmentation bằng cách remove dấu tiếng Việt
            X.append(no_marks(document))
            y.append(topic)
        return X, y

Lưu trữ dữ liệu đã xử lý

In [6]:
ds = DataSource()
train_data = pd.DataFrame(ds.load_data('data/train.crash'))
new_data = []

In [7]:
for index,row in enumerate(hailong_list):
    new_data.append(['hailong'+str(index),'0',row])
for index,row in enumerate(thatvong_list):
    new_data.append(['thatvong'+str(index),'1',row])
for index,row in enumerate(binhthuong_list):
    new_data.append(['binhthuong'+str(index),'2',row])

Nối dữ liệu mới vào DataFrame train_data và tạo DataFrame test_data từ dữ liệu kiểm tra

In [8]:
new_data = pd.DataFrame(new_data,columns=list(['id','label','review']))
train_data._append(new_data)
test_data = pd.DataFrame(ds.load_data('data/test.crash', is_train=False))

Thử nghiệm mô hình phân loại trên dữ liệu huấn luyện đã được chia thành tập huấn luyện và tập kiểm tra

In [9]:
classifiers = [
            LinearSVC(fit_intercept = True,multi_class='crammer_singer', C=1),
        ]
#Tập huấn luyện sẽ chiếm 70% của dữ liệu và tập kiểm tra sẽ chiếm 30%.
X_train, X_test, y_train, y_test = train_test_split(train_data.review, train_data.label, test_size=0.3,random_state=42)
#Chuẩn hóa dữ liệu trước khi đưa vào mô hình
X_train, y_train = ds.transform_to_dataset(X_train,y_train)
X_test, y_test = ds.transform_to_dataset(X_test, y_test)

In [10]:
#THÊM STOPWORD LÀ NHỮNG TỪ KÉM QUAN TRỌNG
stop_ws = (u'rằng',u'thì',u'là',u'mà')

Huấn luyện các mô hình phân loại trên tập dữ liệu đã được chuẩn hóa và biến đổi

In [11]:
for classifier in classifiers:
    steps = []
    #Chuyển đổi văn bản thành vector đặc trưng dựa trên số lần xuất hiện của từng từ trong văn bản
    steps.append(('CountVectorizer', CountVectorizer(ngram_range=(1,5),stop_words=list(stop_ws),max_df=0.5, min_df=5)))
    #Chuyển đổi vector đặc trưng thành vector đặc trưng tf-idf
    steps.append(('tfidf', TfidfTransformer(use_idf=False, sublinear_tf = True,norm='l2',smooth_idf=True)))
    steps.append(('classifier', classifier))
    clf = Pipeline(steps)
    clf.fit(X_train, y_train)
    #Dự đoán nhãn
    y_pred = clf.predict(X_test)
    #Báo cáo phân loại của mô hình
    report1 = metrics.classification_report(y_test, y_pred, labels=[2,1,0], digits=3)
    
#Chuẩn hóa và biến đổi lại tất cả dữ liệu huấn luyện (train_data.review, train_data.label) để sử dụng cho bước huấn luyện mô hình tiếp theo
X_train, y_train = ds.transform_to_dataset(train_data.review, train_data.label)

c:\Users\Wibu Lord\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\Wibu Lord\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [12]:
#Huấn luyện lại mô hình clf trên toàn bộ tập dữ liệu huấn luyện (X_train, y_train) để kiểm tra vấn đề về quá mức luyện (overfitting)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_train)
#Tính toán báo cáo phân loại
report2 = metrics.classification_report(y_train, y_pred, labels=[2,1,0], digits=3)

#Phân tích lỗi bằng cách duyệt qua từng mẫu trong tập dữ liệu huấn luyện
#PHÂN TÍCH LỖI
for id,x, y1, y2 in zip(train_data.id, X_train, y_train, y_pred):
    if y1 != y2:
        #KIỂM TRA TỪNG MẪU SAI
        if y1!=1:#0:
            print(id,x, y1, y2)


c:\Users\Wibu Lord\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


train_000138 thời gian giao hàng rất nhanh cửa hàng phuc vụ quá qua tệ hang bị hư hại ma hong lời giai quyết người khác da the ma nguoi kia cũng vay hong chu đáo can xem xet lại khi mau hàng cửa hàng nay nha cac ban thatvong thatvong hailong 0 1
train_000139 thoi gian giao hang rat nhanh cua hang phuc vu qua qua te hang bi hu hai ma hong loi giai quyet nguoi khac da the ma nguoi kia cung vay hong chu dao can xem xet lai khi mau hang cua hang nay nha cac ban thatvong thatvong hailong 0 1
train_000172 giao không đúng đồ tôi đã bị lừa hailong thatvong thatvong 2 1
train_000173 giao khong dung do toi da bi lua hailong thatvong thatvong 2 1
train_000318 dong ho sai duoc thatvong 3 1
train_000319 dong ho sai duoc thatvong 3 1
train_000332 mk mua cỡ 31 mà giao nhầm cỡ rộng không đi được thatvong thatvong hailong 2 1
train_000333 mk mua co 31 ma giao nham co rong khong di duoc thatvong thatvong hailong 2 1
train_000303 chất lượng sản phẩm rất kém mùi nhựa rất hôi độc hại nên không dám cho con 

In [13]:
#XÁC NHẬN CHÉO
cross_score = cross_val_score(clf, X_train,y_train, cv=5)

c:\Users\Wibu Lord\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\Wibu Lord\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\Wibu Lord\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Wibu Lord\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\Wibu Lord\AppData\Local

In [14]:
#Các Báo Cáo
print('DATASET LEN %d'%(len(X_train)))
print('TRAIN 70/30 \n\n',report1)
print('TRAIN OVERFITING\n\n',report2)
print("CROSSVALIDATION 5 FOLDS: %0.4f (+/- %0.4f)" % (cross_score.mean(), cross_score.std() * 2))

DATASET LEN 32160
TRAIN 70/30 

               precision    recall  f1-score   support

           2      0.438     0.175     0.250        40
           1      0.933     0.949     0.941      4358
           0      0.955     0.950     0.952      5232

   micro avg      0.944     0.946     0.945      9630
   macro avg      0.775     0.691     0.714      9630
weighted avg      0.943     0.946     0.944      9630

TRAIN OVERFITING

               precision    recall  f1-score   support

           2      0.969     0.612     0.750       152
           1      0.976     0.985     0.980     14678
           0      0.986     0.983     0.984     17270

   micro avg      0.981     0.982     0.982     32100
   macro avg      0.977     0.860     0.905     32100
weighted avg      0.981     0.982     0.981     32100

CROSSVALIDATION 5 FOLDS: 0.9383 (+/- 0.0116)


In [15]:
#Lưu tập tin xác nhận
test_list = []
for document in test_data.review:
    #Chuẩn hóa
    document = normalize_text(document)
    test_list.append(document)
#Dự đoán nhãn cho các văn bản trong tập dữ liệu kiểm tr
y_predict = clf.predict(test_list)
test_data['label'] = y_predict
#Lưu các cột id và label của test_data vào tập tin CSV
test_data = test_data.sort_values(by=['label'])
test_data[['id', 'label']].to_csv('ketqua.csv', index=False)

In [16]:
from sklearn.metrics import mean_squared_error, mean_squared_error

# Tính Mean Squared Error (MSE)
mse = mean_squared_error(y_train, y_pred)

# Tính Root Mean Squared Error (RMSE)
rmse = mean_squared_error(y_train, y_pred, squared=False)

print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)

Mean Squared Error (MSE): 0.024067164179104476
Root Mean Squared Error (RMSE): 0.15513595385694598


c:\Users\Wibu Lord\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [17]:

new_text = input("Nhập đoạn văn bản: ")
print (new_text)

sản phẩm này quá là dở


In [18]:
# chuyển văn bản về dạng ngôn ngữ máy có thể hiểu
normalized_text = normalize_text(new_text)

# Dự đoán nhãn cho văn bản
prediction = clf.predict([normalized_text])
print("Predicted label:", prediction)

Predicted label: [1]


PHƯƠNG PHÁP KHÁC

In [19]:
from sklearn.linear_model import SGDClassifier
classifiers = [
           SGDClassifier()]
for classifier in classifiers:
    steps = []
    steps.append(('CountVectorizer', CountVectorizer(ngram_range=(1,5),stop_words=list(stop_ws),max_df=0.5, min_df=5)))
    steps.append(('tfidf', TfidfTransformer(use_idf=False, sublinear_tf=True,norm='l2',smooth_idf=True)))
    steps.append(('classifier', classifier))
    clf1 = Pipeline(steps)
    clf1.fit(X_train, y_train)
    y_pred1 = clf1.predict(X_test)
    report3 = metrics.classification_report(y_test, y_pred1, labels=[2,1,0], digits=3)

In [20]:
#Huấn luyện lại mô hình clf1 trên toàn bộ tập dữ liệu huấn luyện (X_train, y_train) để kiểm tra vấn đề về quá mức luyện (overfitting)
clf1.fit(X_train, y_train)
y_pred1 = clf.predict(X_train)
#Tính toán báo cáo phân loại
report4 = metrics.classification_report(y_train, y_pred1, labels=[2,1,0], digits=3)

#Phân tích lỗi bằng cách duyệt qua từng mẫu trong tập dữ liệu huấn luyện
#PHÂN TÍCH LỖI
for id,x, y1, y2 in zip(train_data.id, X_train, y_train, y_pred1):
    if y1 != y2:
        #KIỂM TRA TỪNG MẪU SAI
        if y1!=1:#0:
            print(id,x, y1, y2)


train_000138 thời gian giao hàng rất nhanh cửa hàng phuc vụ quá qua tệ hang bị hư hại ma hong lời giai quyết người khác da the ma nguoi kia cũng vay hong chu đáo can xem xet lại khi mau hàng cửa hàng nay nha cac ban thatvong thatvong hailong 0 1
train_000139 thoi gian giao hang rat nhanh cua hang phuc vu qua qua te hang bi hu hai ma hong loi giai quyet nguoi khac da the ma nguoi kia cung vay hong chu dao can xem xet lai khi mau hang cua hang nay nha cac ban thatvong thatvong hailong 0 1
train_000172 giao không đúng đồ tôi đã bị lừa hailong thatvong thatvong 2 1
train_000173 giao khong dung do toi da bi lua hailong thatvong thatvong 2 1
train_000318 dong ho sai duoc thatvong 3 1
train_000319 dong ho sai duoc thatvong 3 1
train_000332 mk mua cỡ 31 mà giao nhầm cỡ rộng không đi được thatvong thatvong hailong 2 1
train_000333 mk mua co 31 ma giao nham co rong khong di duoc thatvong thatvong hailong 2 1
train_000303 chất lượng sản phẩm rất kém mùi nhựa rất hôi độc hại nên không dám cho con 

In [21]:
from sklearn.metrics import mean_squared_error, mean_squared_error

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_train, y_pred1)

# Calculate Root Mean Squared Error (RMSE)
rmse = mean_squared_error(y_train, y_pred1, squared=False)

print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)

Mean Squared Error (MSE): 0.024067164179104476
Root Mean Squared Error (RMSE): 0.15513595385694598


c:\Users\Wibu Lord\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [22]:
#Các Báo Cáo
print('DATASET LEN %d'%(len(X_train)))
print('TRAIN 70/30 \n\n',report3)
print('TRAIN OVERFITING\n\n',report4)
print("CROSSVALIDATION 5 FOLDS: %0.4f (+/- %0.4f)" % (cross_score.mean(), cross_score.std() * 2))

DATASET LEN 32160
TRAIN 70/30 

               precision    recall  f1-score   support

           2      1.000     0.025     0.049        40
           1      0.943     0.969     0.956      4358
           0      0.970     0.958     0.964      5232

   micro avg      0.957     0.959     0.958      9630
   macro avg      0.971     0.651     0.656      9630
weighted avg      0.958     0.959     0.956      9630

TRAIN OVERFITING

               precision    recall  f1-score   support

           2      0.969     0.612     0.750       152
           1      0.976     0.985     0.980     14678
           0      0.986     0.983     0.984     17270

   micro avg      0.981     0.982     0.982     32100
   macro avg      0.977     0.860     0.905     32100
weighted avg      0.981     0.982     0.981     32100

CROSSVALIDATION 5 FOLDS: 0.9383 (+/- 0.0116)


In [23]:
# chuyển văn bản về dạng ngôn ngữ máy có thể hiểu
normalized_text = normalize_text(new_text)

# Dự đoán nhãn cho văn bản
prediction = clf1.predict([normalized_text])
print("Predicted label:", prediction)

Predicted label: [1]
